# Introduction

This Notebooks is a join notebook from both the prepare_data and pytorch-bst in order to be run in google colab.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Prepare data section

In [ ]:
!pip install pytorch_lightning torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.2/616.2 kB 54.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 22.7 MB/s eta 0:00:00
  Created wheel for torch_geometric: filename=torch_geometric-2.3.0-py3-none-any.whl size=909897 sha256=a9b47fcb73c8427e2d401b6cd7a5788a34c0ccbf1a332cdbeaabf50531548383
  Stored in directory: /root/.ca

In [ ]:
import pandas as pd
import torch
# import pytorch_lightning as pl
from tqdm import tqdm
import torchmetrics
import torch_geometric
import math
from urllib.request import urlretrieve
from zipfile import ZipFile
import os
import torch.nn as nn
import numpy as np
from math import sqrt
import pickle

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())

True


## Data

In [ ]:
urlretrieve('http://files.grouplens.org/datasets/movielens/ml-1m.zip', 'movielens.zip')
ZipFile('movielens.zip', 'r').extractall()

In [ ]:
ratings = pd.read_csv(
    'ml-1m/ratings.dat',
    sep='::',
    names=['user_id', 'movie_id', 'rating', 'unix_timestamp'],
)

<ipython-input-82-9b1c39698ca2>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv(


In [ ]:
# ratings['unix_timestamp'] = pd.to_datetime(ratings['unix_timestamp'],unit='s')
ratings['movie_id'] = ratings['movie_id'].astype(int)
ratings['user_id'] = ratings['user_id'].astype(int)
ratings = ratings.sort_values(by='unix_timestamp')
ratings

,user_id,movie_id,rating,unix_timestamp
1000138,6040,858,4,956703932
1000153,6040,2384,4,956703954
999873,6040,593,5,956703954
1000007,6040,1961,4,956703977
1000192,6040,2019,5,956703977
...,...,...,...,...
825793,4958,2399,1,1046454338
825438,4958,1407,5,1046454443
825724,4958,3264,4,1046454548
825731,4958,2634,3,1046454548


In [ ]:
# save primary csv's
if not os.path.exists('data'):
    os.makedirs('data')
    
ratings.to_csv('data/ratings.csv',index=False)

In [ ]:
users = pd.read_csv("./ml-1m/users.dat", sep="::", names = ['userID','gender','age','occupation','zipcode'])
users.head()

<ipython-input-145-84a6d861c214>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_csv("./ml-1m/users.dat", sep="::", names = ['userID','gender','age','occupation','zipcode'])


,userID,gender,age,occupation,zipcode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [ ]:
from torch_geometric.data import Data
from torch_geometric.utils import to_dense_adj

ratings_std = ratings.copy(deep=True)
ratings_std['movie_id'] = ratings['movie_id'] + ratings['user_id'].max()
ratings_std['rating'] = ratings['rating']

edge_list = [list(ratings_std['user_id']), list(ratings_std['movie_id'])]
edge_index = torch.tensor(edge_list, dtype=torch.long)

adj_list = to_dense_adj(edge_index)[0]
ratings_list = torch.tensor(list(ratings_std['rating']), dtype=torch.float)

dataset = Data(x=adj_list, y=ratings_list, edge_index=edge_index)

train_num = int(dataset.num_edges * 0.8)
val_num = int(dataset.num_edges * 0.1)
test_num = dataset.num_edges - train_num - val_num
dataset.train_mask = torch.cat((torch.ones(train_num, 1, dtype=torch.int), torch.zeros(val_num + test_num, 1, dtype=torch.int)))
dataset.val_mask = torch.cat((torch.zeros(train_num, 1, dtype=torch.int), torch.ones(val_num, 1, dtype=torch.int), torch.zeros(test_num, 1, dtype=torch.int)))
dataset.test_mask = torch.cat((torch.zeros(train_num + val_num, 1, dtype=torch.int), torch.ones(test_num, 1, dtype=torch.int)))

data = dataset.to(device)

print(dataset.num_node_features, dataset.num_nodes, dataset.num_edges)

9993 9993 1000209


In [ ]:
ratings.iloc[train_num+val_num]

user_id                  24
movie_id               2683
rating                    4
unix_timestamp    978133367
Name: 3210, dtype: int64

In [ ]:
train_num, val_num, test_num

(800167, 100020, 100022)

In [ ]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN_NGCF(torch.nn.Module):
    def __init__(self, dataset, hidden_channels_gcn=32, hidden_channels_fc=512, dropout_probability=0.2):
        super().__init__()
        self.dataset = dataset
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels_gcn)
        self.conv2 = GCNConv(hidden_channels_gcn, hidden_channels_gcn)
        self.dropout_p = dropout_probability
        self.fc1 = nn.Linear(in_features=(2 * hidden_channels_gcn), out_features=hidden_channels_fc)
        self.fc2 = nn.Linear(in_features=hidden_channels_fc, out_features=hidden_channels_fc)
        self.output = nn.Linear(in_features=hidden_channels_fc, out_features=1)
        self.relu = nn.ReLU()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        out = self.conv1(x, edge_index)
        out = F.relu(out)
        out = F.dropout(out, p=self.dropout_p, training=self.training)
        out = self.conv2(out, edge_index)
        out = F.relu(out)
        out = F.dropout(out, p=self.dropout_p, training=self.training)

        out = self.fc1(torch.cat((out[data.edge_index.T[:, 0]], out[data.edge_index.T[:, 1]]), dim=1).type(torch.float))
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.output(out)
        # out = self.relu(out)

        return out

In [ ]:
model = GCN_NGCF(data, hidden_channels_gcn=18, hidden_channels_fc=128, dropout_probability=0.2).to(device)
print(model)

criterion = nn.MSELoss()
mae_loss = nn.L1Loss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00005, weight_decay=1e-6)

training_rmse_vals = []
test_rmse_vals = []
val_mae_vals = []
test_mae_vals = []

num_epochs = 51
data = data.to(device)

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    out = model(data).flatten()

    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    training_rmse = loss.detach().cpu()

    model.eval()
    with torch.no_grad():
      out = model(data).flatten()
      val_rmse = criterion(out[data.val_mask], data.y[data.val_mask]).detach().cpu()
      test_rmse = criterion(out[data.test_mask], data.y[data.test_mask]).detach().cpu()
      val_mae = mae_loss(out[data.val_mask], data.y[data.val_mask]).detach().cpu()
      test_mae = mae_loss(out[data.test_mask], data.y[data.test_mask]).detach().cpu()

    training_rmse_vals.append(training_rmse)
    test_rmse_vals.append(test_rmse)
    val_mae_vals.append(val_mae)
    test_mae_vals.append(test_mae)

    # if epoch % 5 == 0:
    file_path = f"/content/drive/MyDrive/WSTM_latest/weights/ngcf_latest_{epoch}.pth"
    torch.save(model.state_dict(), file_path)
    print(f'Epoch {epoch}, Training RMSE: {training_rmse}, Val RMSE: {val_rmse}')

GCN_NGCF(
  (conv1): GCNConv(9993, 18)
  (conv2): GCNConv(18, 18)
  (fc1): Linear(in_features=36, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
)
Epoch 0, Training RMSE: 14.287375450134277, Val RMSE: 11.823831558227539
Epoch 1, Training RMSE: 12.777234077453613, Val RMSE: 11.36196517944336
Epoch 2, Training RMSE: 11.825504302978516, Val RMSE: 10.865215301513672
Epoch 3, Training RMSE: 11.929028511047363, Val RMSE: 10.423948287963867
Epoch 4, Training RMSE: 11.298694610595703, Val RMSE: 9.937043190002441
Epoch 5, Training RMSE: 11.855331420898438, Val RMSE: 9.45305061340332
Epoch 6, Training RMSE: 10.163991928100586, Val RMSE: 8.93957233428955
Epoch 7, Training RMSE: 9.277521133422852, Val RMSE: 8.400430679321289
Epoch 8, Training RMSE: 11.1415433883667, Val RMSE: 7.872442722320557
Epoch 9, Training RMSE: 9.63935375213623, Val RMSE: 7.328922748565674
Epoch 10, Tra

In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/WSTM_latest/weights/ngcf_latest.pth")

In [ ]:
stored_model = GCN_NGCF(data, hidden_channels_gcn=18, hidden_channels_fc=128, dropout_probability=0.2)
stored_model.load_state_dict(torch.load("/content/drive/MyDrive/WSTM_latest/weights/ngcf_latest_50.pth"))

<All keys matched successfully>

In [ ]:
ngcf_emb = {}

In [ ]:
def get_middle_layer_output(model, input, output):
    middle_layer_output = output.detach().clone() # Get output from the last layer
    ngcf_emb['bb'] = middle_layer_output
    print(middle_layer_output)
    # return middle_layer_output

# Register the hook to the second layer
hook = model.conv2.register_forward_hook(get_middle_layer_output)
out = model(data)
# hook.remove()

In [ ]:
import pickle
pickle.dump(ngcf_emb['bb'], open("ngcf_emb.pkl", "wb"))